In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# create empty dataframe
df = pd.DataFrame(columns=['Date', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 'Winner', 'Bubble'])

# list of dates
dates = pd.date_range(start='2013-10-05', end='2020-10-11')

# date NBA started playing in bubble
start_bubble = pd.to_datetime('2020-07-22')

In [ ]:
# loop through dates and scrape data from games
for date in dates:
    # fetch page for given date
    resp = requests.get('https://www.nba.com/games?date={}'.format(date))
    soup = BeautifulSoup(resp.content)
    # store data from all games on date in list
    games = soup.find_all('a', class_='flex-1 px-2 pt-5 h-full block hover:no-underline relative text-sm pt-5 pb-4 mb-1 px-2')
    # check if any games were played
    if len(games) == 0:
        continue
    else:
        # loop through games and gather teams and score data
        for game in games:
            d = {}
            teams = [g.get_text() for g in game.find_all('span', class_='relative')]
            scores = [s.get_text() for s in game.find_all('p', class_='h9 relative inline-block leading-none')]
            d['Date'] = date
            try:
                d['Home Team'] = teams[1]
                d['Away Team'] = teams[0]
                d['Home Score'] = scores[1]
                d['Away Score'] = scores[0]
                if date >= start_bubble:
                    d['Bubble'] = 'Y'
                else:
                    d['Bubble'] = 'N'
                if scores[1] > scores[0]:
                    d['Winner'] = 'A'
                else:
                    d['Winner'] = 'H'
                df = df.append(d, ignore_index=True)
            except:
                pass

In [ ]:
 df.to_csv('nba_stats.csv')